Yael Brown<br>
October 31, 2024
## Dataset CombinerDA¶
This notebook is used combining the fraud. transaction data, price data, and normal transaction data into one dataset. del.

In [12]:
# Load libraries
import pandas as pd
import os
import math
import time
import warnings

In [3]:
# Suppress warnings in the notebook
warnings.filterwarnings("ignore")

In [10]:
# Define constants
BTC_DATA_PATH = '../data/btc_trans_data/data_raw/'

In [53]:
btc_trans_filenames = os.listdir(BTC_DATA_PATH)

btc_trans_filenames.remove('.ipynb_checkpoints')
btc_trans_filenames.sort()

print(btc_trans_filenames)

['5626370', '5626371', '5626372', '5626373', '5626374', '5626375', '5626376', '5626377', '5626378', '5626379', '5626380', '5626381', '5626382', '5626383', '5626384', '5626385', '5626386', '5626387', '5626388', '5626389', '5626390', '5626391', '5626392', '5626393', '5626394', '5626395', '5626396', '5626397', '5626398', '5626399', '5626400', '5626401', '5626402', '5626403', '5626404', '5626405', '5626406', '5626407', '5626408', '5626409', '5626410', '5626411', '5626412', '5626413', '5626414', '5626415', '5626416', '5626417', '5626418', '5626419', '5626420', '5626421', '5626422', '5626423', '5626424', '5626425', '5626426', '5626427', '5626428', '5626429', '5626430', '5626431', '5626432', '5626433', '5626434', '5626435', '5626436', '5626437', '5626438', '5626439', '5626440', '5626441', '5626442', '5626443', '5626444', '5626445', '5626446', '5626447', '5626448', '5626449', '5626450', '5626451', '5626452', '5626453', '5626454', '5626455', '5626456', '5626457', '5626458', '5626459', '5626460'

In [55]:
# Method for return year from timestamp
def get_year(row):
    ts = time.gmtime(row['timestamp'])
    return ts.tm_year

In [70]:
# Check which files are within the year range
in_year_range = []

cnt = 0
for file in btc_trans_filenames: 
    try: 
        isWithinRange = False
        df = pd.read_csv(BTC_DATA_PATH + file)
        df.sort_values(by=['timestamp'])
        
        start_year = get_year(df.iloc[0])
        end_year = get_year(df.iloc[-1])
        
        if start_year >= 2016 and end_year <= 2018:
            in_year_range.append(file)
            isWithinRange = True
        
        print(f"{cnt}/{len(btc_trans_filenames)}: {isWithinRange}    start: {start_year} end: {end_year}")
    except Exception as e:
        print(f"Error with file: {file}\n{e}")

    cnt += 1

0/289: False    start: 2009 end: 2009
1/289: False    start: 2009 end: 2009
2/289: False    start: 2009 end: 2009
3/289: False    start: 2009 end: 2009
4/289: False    start: 2009 end: 2009
5/289: False    start: 2009 end: 2009
6/289: False    start: 2009 end: 2009
7/289: False    start: 2009 end: 2009
8/289: False    start: 2009 end: 2009
9/289: False    start: 2009 end: 2009
10/289: False    start: 2009 end: 2009
11/289: False    start: 2009 end: 2009
12/289: False    start: 2010 end: 2010
13/289: False    start: 2010 end: 2010
14/289: False    start: 2010 end: 2010
15/289: False    start: 2010 end: 2010
16/289: False    start: 2010 end: 2010
17/289: False    start: 2010 end: 2010
18/289: False    start: 2010 end: 2010
19/289: False    start: 2010 end: 2010
20/289: False    start: 2010 end: 2010
21/289: False    start: 2010 end: 2010
22/289: False    start: 2010 end: 2010
23/289: False    start: 2010 end: 2010
24/289: False    start: 2011 end: 2011
25/289: False    start: 2011 end: 2

In [72]:
# Show files within range
print(in_year_range)
print(len(in_year_range))

['5626480', '5626483', '5626486', '5626489', '5626492', '5626495', '5626498', '5626500', '5626502', '5626504', '5626506', '5626508', '5626511', '5626514', '5626517', '5626520', '5626527', '5626530', '5626534', '5626536', '5626537', '5626541', '5626545', '5626549', '5626555', '5626565', '5626584', '5626587', '5626591', '5626594', '5626597', '5626600', '5626603', '5626604', '5626606', '5626609', '5626616', '5626617', '5626620', '5626667', '5635409', '5635413', '5635417']
43


In [ ]:
# Save this list because it took awhile to compute
in_year_range_save = ['5626480', '5626483', '5626486', '5626489', '5626492', '5626495', '5626498', '5626500', '5626502', '5626504', '5626506', '5626508', '5626511', '5626514', '5626517', '5626520', '5626527', '5626530', '5626534', '5626536', '5626537', '5626541', '5626545', '5626549', '5626555', '5626565', '5626584', '5626587', '5626591', '5626594', '5626597', '5626600', '5626603', '5626604', '5626606', '5626609', '5626616', '5626617', '5626620', '5626667', '5635409', '5635413', '5635417']

In [74]:
# Process and combine only files that are within range
cnt = 0 

for file in in_year_range:
    print(f"File: {file}") 
    # Read file
    df = pd.read_csv(BTC_DATA_PATH + file)
    
    # DropNA's
    df.dropna(inplace=True)

    # Create columns in dataframe for month, year, and day
    df['year'], df['month'], df['day'] = 0,0,0

    # Populate month, day, year for each row
    idx = 0
    tot_len = len(df)

    for r in range(0,tot_len): 
        try:
            ts = time.gmtime(int(df.loc[r, 'timestamp']))
            df.loc[r, 'year'] = ts.tm_year
            df.loc[r, 'month'] = ts.tm_mon
            df.loc[r, 'day'] = ts.tm_mday
            print(f"{idx}/{tot_len}/{file}")
        except Exception as e:
            df.loc[r, 'year'] = None
            df.loc[r, 'month'] = None
            df.loc[r, 'day'] = None
            print("Error for idx: ", idx)
            print(e)
        finally:
            idx += 1

    # Dropna again if there are errors processing rows
    df.dropna(inplace=True)

    # Save File and add process layer. 
    df.to_csv((BTC_DATA_PATH + f"{file}_processed.csv"), index=False, header=True, sep=',')

File: 5626480
0/24140322/5626480
1/24140322/5626480
2/24140322/5626480
3/24140322/5626480
4/24140322/5626480
5/24140322/5626480
6/24140322/5626480
7/24140322/5626480
8/24140322/5626480
9/24140322/5626480
10/24140322/5626480
11/24140322/5626480
12/24140322/5626480
13/24140322/5626480
14/24140322/5626480
15/24140322/5626480
16/24140322/5626480
17/24140322/5626480
18/24140322/5626480
19/24140322/5626480
20/24140322/5626480
21/24140322/5626480
22/24140322/5626480
23/24140322/5626480
24/24140322/5626480
25/24140322/5626480
26/24140322/5626480
27/24140322/5626480
28/24140322/5626480
29/24140322/5626480
30/24140322/5626480
31/24140322/5626480
32/24140322/5626480
33/24140322/5626480
34/24140322/5626480
35/24140322/5626480
36/24140322/5626480
37/24140322/5626480
38/24140322/5626480
39/24140322/5626480
40/24140322/5626480
41/24140322/5626480
42/24140322/5626480
43/24140322/5626480
44/24140322/5626480
45/24140322/5626480
46/24140322/5626480
47/24140322/5626480
48/24140322/5626480
49/24140322/5626

KeyboardInterrupt: 

In [ ]:
# Combine processed datasets into one dataset
btc_trans_processed_filenames = os.listdir(BTC_DATA_PATH)
btc_trans_processed_filenames.remove('.ipynb_checkpoints')

temp = []
for name in btc_trans_processed_filenames:
    if '_processed' in name:
        temp.append(name)
    else: 
        continue

btc_trans_processed_filenames = temp
del temp

In [ ]:
# Combine datasets into one dataset. 
out_df = pd.DataFrame()

cnt = 0
processed_length = len(btc_trans_processed_filenames)
for file in btc_trans_processed_filenames: 
    # Read in files
    df = pd.read_csv(BTC_DATA_PATH + file)
    
    # Concat with out_df
    out_df = pd.concat([out_df, df])

# Save output file
df.to_csv((BTC_DATA_PATH + f"btc_transactions_final.csv"), index=False, header=True, sep=',')

In [76]:
24000000 * 43

1032000000

In [37]:
# Copy import
import pandas as pd # add import to make it easier to run the following cells when the kernal crashes. 

df = pd.read_csv(BTC_DATA_PATH + '5626486')

df.head()

,timestamp,source_address,destination_address,satoshi
0,1456791197,16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC,1KgqD2G2NSZYgHk5m7AW3wxupDj3H76Q8e,156555.0
1,1456791197,16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC,1NZF5BEn5iCe655fj4vd5UMaXXfVvFH5aJ,26400000.0
2,1456791197,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,1NXmUs99Rn8FzyYMK5bmBAPL81VzmPhH9d,245057610.0
3,1456791197,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,48151.0
4,1456791197,1PaQM5ngUGzZYS3e9epbY7BiZvrukDoJQB,14jvp9Sj4peh28CKxifLob2jsZAcYPfZ74,20860000.0


In [72]:
heist_set = pd.read_csv("heist_set_addresses.csv")

In [88]:
print("Creating source set")
df_source_set = set(df['source_address'])
print("Creating Destination set")
df_dest_set = set(df['destination_address'])

print("Combining sets")
df_addresses_set = df_source_set.union(df_dest_set)

print("Finding matches")
all_matches = df_addresses_set.intersection(set(heist_set['0']))

Creating source set
Creating Destination set
Combining sets
Finding matches


In [90]:
print(len(all_matches))

681


In [92]:
all_matches

{'112GocBgFSnaote6krx828qaockFraD8mp',
 '11445ybEvoBuFCnaYGni4KQDgKRZ3huiJX',
 '11UC4DMzTR1qQnNEHs9XpotWYAmzs6nBM',
 '11UUivRcVdRJWymipDPB6WzbxzWmL7a3B',
 '121D2DAFrRGLHoeZDaAT1rjEYj69EknvsT',
 '123zTvjw931gKezg4bMwaBS45YBQjJh3pZ',
 '12KXG7xgpBram9UhSeKRs4CrCmSoUZdvSY',
 '12Quexr6wiohQ62PDwLuGxAJrqWafQYP2X',
 '12Rn1YeBYRB2HNpcgUD4BGH39PC8pNPTta',
 '12Ro82Eh7HiZbk8McYEsC82qe4p53e79LV',
 '12Rq32tkAxZdFm1EFfqzpDhH5xXY7dSHgN',
 '12SfsswWdXL1LFhTaVh8pKWcgtDRkumWmL',
 '12UJgTp4msKCK41d9xCv5Bo37p1iRQbLqh',
 '12Uz7CaY34ysuMeRdctEzsyoKqyyCJda62',
 '12Vkffq8BVufN1noGWpvyZNC7Fd5dnUVP2',
 '12YWHNUSKt241MHAkXG8RtATcDRiDTkApc',
 '12aYrgSNW38bbYQ11oJiCi97g945f2rUgE',
 '12bh99VY4p4nmtCv5d7A6eXbznhauPjYut',
 '12e5Lb5i3m3vEwmJMNegyHSv1FeEusYjUA',
 '12ePRqvoRkY5biVFPyp6beExAaWiaZWYhm',
 '12ebZCSFvPvkfaHo5E3rQgkQMBvn1ccPK2',
 '12gZcVcwhYD9dabcF8UakAmHQbH37ZkdKr',
 '12iY6mnQQ6f2F9Tdzo6pC37fbwynqavGYP',
 '12kVzMMhb7DGfj2vfiq1Sm6dUgxygMRFAC',
 '12kgp69XXHpsu7tDyBRwEMQqE6hs9dnWZV',
 '12nkP9tRsabG2wiAXVdXK6aZa

In [94]:
len(df)

24104551

In [97]:
temp_df = df[df['source_address'].isin(all_matches) | df['destination_address'].isin(all_matches)]

print(len(temp_df))

2733


## A lot of transactions
Since its estimated to be over a billion lines, a smaller subset from a 2016 dataset will be taken to complete EDA. 

In [14]:
import pandas as pd # add import to make it easier to run the following cells when the kernal crashes. 

df = pd.read_csv(BTC_DATA_PATH + '5626486')

df.head()

,timestamp,source_address,destination_address,satoshi
0,1456791197,16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC,1KgqD2G2NSZYgHk5m7AW3wxupDj3H76Q8e,156555.0
1,1456791197,16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC,1NZF5BEn5iCe655fj4vd5UMaXXfVvFH5aJ,26400000.0
2,1456791197,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,1NXmUs99Rn8FzyYMK5bmBAPL81VzmPhH9d,245057610.0
3,1456791197,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,48151.0
4,1456791197,1PaQM5ngUGzZYS3e9epbY7BiZvrukDoJQB,14jvp9Sj4peh28CKxifLob2jsZAcYPfZ74,20860000.0


In [99]:
# Initial shape
df.shape

(24104551, 4)

In [101]:
print("Beginning Shape: ", df.shape)

df = df.iloc[:math.floor(df.shape[0] * .004)]

print("Ending Shape: ", df.shape)

Beginning Shape:  (24104551, 4)
Ending Shape:  (96418, 4)


In [103]:
# Cheat: Merge rows from list of known fraudulent addresses in them with sampled dataset
df = pd.concat([df, temp_df])

# Drop duplicates
df = df.drop_duplicates()

# Reset index
df = df.reset_index(drop=True)

In [105]:
# Check size again
df.shape

(80120, 4)

In [107]:
# Verify if there are na's
df.isna().sum()

timestamp              0
source_address         0
destination_address    0
satoshi                0
dtype: int64

In [109]:
#dropna's
df.dropna(inplace=True)

# Verify if there are na's
print(df.isna().sum())

timestamp              0
source_address         0
destination_address    0
satoshi                0
dtype: int64


In [111]:
# Create columns in dataframe for month, year, and day
df['year'], df['month'], df['day'] = 0,0,0

In [113]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80120 entries, 0 to 80119
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   timestamp            80120 non-null  int64  
 1   source_address       80120 non-null  object 
 2   destination_address  80120 non-null  object 
 3   satoshi              80120 non-null  float64
 4   year                 80120 non-null  int64  
 5   month                80120 non-null  int64  
 6   day                  80120 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 4.3+ MB
None
    timestamp                      source_address  \
0  1456791197  16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC   
1  1456791197  16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC   
2  1456791197  1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo   
3  1456791197  1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo   
4  1456791197  1PaQM5ngUGzZYS3e9epbY7BiZvrukDoJQB   

                  destination_address      satoshi  year

In [115]:
idx = 0
tot_len = len(df)

for r in range(0,tot_len): 
    try:
        ts = time.gmtime(int(df.loc[r, 'timestamp']))
        df.loc[r, 'year'] = ts.tm_year
        df.loc[r, 'month'] = ts.tm_mon
        df.loc[r, 'day'] = ts.tm_mday
    except Exception as e:
        df.loc[r, 'year'] = None
        df.loc[r, 'month'] = None
        df.loc[r, 'day'] = None
        print("Error for idx: ", idx)
        print(e)
    finally:
        if idx % 100 == 0: print(f"{idx}/{tot_len}")
        idx += 1

0/80120
100/80120
200/80120
300/80120
400/80120
500/80120
600/80120
700/80120
800/80120
900/80120
1000/80120
1100/80120
1200/80120
1300/80120
1400/80120
1500/80120
1600/80120
1700/80120
1800/80120
1900/80120
2000/80120
2100/80120
2200/80120
2300/80120
2400/80120
2500/80120
2600/80120
2700/80120
2800/80120
2900/80120
3000/80120
3100/80120
3200/80120
3300/80120
3400/80120
3500/80120
3600/80120
3700/80120
3800/80120
3900/80120
4000/80120
4100/80120
4200/80120
4300/80120
4400/80120
4500/80120
4600/80120
4700/80120
4800/80120
4900/80120
5000/80120
5100/80120
5200/80120
5300/80120
5400/80120
5500/80120
5600/80120
5700/80120
5800/80120
5900/80120
6000/80120
6100/80120
6200/80120
6300/80120
6400/80120
6500/80120
6600/80120
6700/80120
6800/80120
6900/80120
7000/80120
7100/80120
7200/80120
7300/80120
7400/80120
7500/80120
7600/80120
7700/80120
7800/80120
7900/80120
8000/80120
8100/80120
8200/80120
8300/80120
8400/80120
8500/80120
8600/80120
8700/80120
8800/80120
8900/80120
9000/80120
9100/80120


In [117]:
# Verify the date 
df.head(10)

,timestamp,source_address,destination_address,satoshi,year,month,day
0,1456791197,16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC,1KgqD2G2NSZYgHk5m7AW3wxupDj3H76Q8e,1.565550e+05,2016,3,1
1,1456791197,16ZZav1Cq5STgj7X5bvP9UgVYJkRk2RDdC,1NZF5BEn5iCe655fj4vd5UMaXXfVvFH5aJ,2.640000e+07,2016,3,1
2,1456791197,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,1NXmUs99Rn8FzyYMK5bmBAPL81VzmPhH9d,2.450576e+08,2016,3,1
3,1456791197,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,1NcktapNVSfbH9FZ9mWZcgMaaokNj9Goyo,4.815100e+04,2016,3,1
4,1456791197,1PaQM5ngUGzZYS3e9epbY7BiZvrukDoJQB,14jvp9Sj4peh28CKxifLob2jsZAcYPfZ74,2.086000e+07,2016,3,1
5,1456791197,1HTiUfiEcYt4onvdcVKxqyxQbYYKZRM1fh,13DMmdw7rvCn82X5VXmgpQrZTYEjQovS1Y,1.239000e+07,2016,3,1
6,1456791197,18Dszthc4jyqZC5DkxMEPpV9BVs2yjw2y6,1HUduEj16uVCN3KRw9CkezXqCJhpTrFYVk,1.199000e+08,2016,3,1
7,1456791197,1B8Y6DggGWSqYMKDqbMaWp9C515BwgjsL2,1KX54PVSejEsVgzEB3EQuwc3QLovsRGyC4,1.031947e+09,2016,3,1
8,1456791197,1AUFqg17oasCP4pEodbrbUBrh7R8BHvVnS,1NKvdvgqJLiG98F6Em78WWe1Ayv1kSkgnc,1.390000e+07,2016,3,1
9,1456791197,16gXssjFn9yrGv5ooiapzA438DMEbZhLbY,1DmgEkFeeeBCJHY1sad2qQ1ymGNiQWgwD8,4.099263e+08,2016,3,1


In [119]:
# Verify no na's
df.isna().sum()

timestamp              0
source_address         0
destination_address    0
satoshi                0
year                   0
month                  0
day                    0
dtype: int64

In [121]:
# Write output CSV for modeling
df.to_csv((BTC_DATA_PATH + "btc_data_cleaned.csv"), index=False, header=True, sep=',')